# Firmata: Control de microcontroladores por puerto serial
Firmata es un protocolo de comunicaciones Open Source entre una aplicación de software y microcontroladres. El protocolo puede ser implementado en firmware en cualquier arquitectura de un microcontrolador o por software. [El codigo fuente del protocolo puede ser consultado aqui](https://github.com/firmata/protocol), así como las versiones cliente para diferentes lenguajes de programación.

Aunque el protocolo Firmata puede ser implementado en cualquier microcontolador, actualmente la implemetación más completa esta disponible para Arduino (incluendo las versiones compatibles). 

Para contolar un tarjeta de desarrollo Arduino utilizando Python se debe de instalar alguno de los clientes Firmata disponibles. El más popular es [pyFirmata](https://github.com/tino/pyFirmata):

    pip install pyfirmata
    
Se debe de tener la placa Arduino con el script `StandardFirmata` subido (Archivo > Ejemplos > Firmata).

![](https://electropeak.com/learn/wp-content/uploads/2019/07/rpiard-14.compressed.jpg)

Una aplicación del uso de este protocolo es para la interconexión de un RaspberryPi con un Arduino y aprovechar este último como una tarjeta de adquisición de datos, pues el RPi no tiene un pin analógico ni las capacidades ADC del modulo Arduino.

## Controlando pines desde Python
El código para escribir sobre un pin digital en Arduino desde el RPi utilizando Firmata es el siguiente:

In [ ]:
import pyfirmata

# Pin integrado en el Modulo Arduino
led_pin = 13

# Conexion al puerto serial USB
board = pyfirmata.Arduino("/dev/ttyUSB0")
print("Code is running")

# Lazo de control para acceder a un pin
while True:
    board.digital[led_pin].write(0)
    board.pass_time(1)
    board.digital[led_pin].write(1)
    board.pass_time(1)

board.exit()

## Leyendo el puerto analógico de Arduino
Se lee un puerto analógico A0 en Arduino:

In [ ]:
from pyfirmata import Arduino, util
import time

# Se establece la conexion con el modulo Arduino
print("Connecting to Arduino board. Please wait...")
board = Arduino('/dev/ttyUSB0')

# Se genera un iterador para el envio de datos
it = util.Iterator(board)
it.start()
time.sleep(0.5)

# Se establece el monitoreo del puerto analogico
print("Arduino Connected\n")
board.analog[1].enable_reporting()

# Especificacion del pin de trabajo (tipo <a|d>:numero:in/out <i|o>)
a0 = board.get_pin('a:1:i')

# Lazo de lectura del pin analogico
while True:
    print("Analog Value:{}".format(a0.read()))
    time.sleep(1)

board.exit()

## Lectura de un Joystick analógico en Arduino desde RPi
El siguiente script debe de estar en un Raspberry Pi con un modulo Arduino conectado por puerto serial.

Debe tener un Joystick instalado:

    JOYSTICK    ARDUINO
      VRX   --->   A1
      VRY   --->   A2
      GND   --->   GND
      +5V   --->   5V

In [ ]:
from pyfirmata import Arduino, util
import time

print("Connecting to Arduino board. Please wait...")
board = Arduino('/dev/ttyUSB0')
it = util.Iterator(board)
it.start()

print("Arduino Connected\n")
board.analog[1].enable_reporting()
board.analog[2].enable_reporting()

a0 = board.get_pin('a:1:i')
a1 = board.get_pin('a:2:i')

while True:
    a0_val = a0.read()
    a1_val = a1.read()

    # Control de rangos para deteccion del movimiento
    if a0_val is not None:
        if a0.read() < 0.3:
            print("Izquierda")
        elif a0.read() > 0.8:
            print("Derecha")

    if a1_val is not None:
        if a1.read() < 0.3:
            print("Abajo")
        elif a1.read() > 0.8:
            print("Arriba")
        
    time.sleep(0.5)

board.exit()

## Aplicacion: Joystick de juego en RPi (pygame)

In [ ]:
from pyfirmata import Arduino, util
import time
import pygame, sys
from pygame.locals import *

WHITE = (255, 255, 255)
RED = (255, 0, 0)
HEIGHT = 320
WIDTH = 240
SIDE = 10

print("Connecting to joystick. Please wait...")
board = Arduino('/dev/ttyUSB0')
it = util.Iterator(board)
it.start()
time.sleep(0.5)

print("Joystick connected\n")
board.analog[1].enable_reporting()
board.analog[2].enable_reporting()

a0 = board.get_pin('a:1:i')
a1 = board.get_pin('a:2:i')

# Inicializacion de pygame (reloj, pantalla)
pygame.init()
clock = pygame.time.Clock()
screen = pygame.display.set_mode((WIDTH, HEIGHT))

# Posicion del cursor
posx, posy = 0, 0

# Lazo del juego
while True:
    # Condicion del salida del juego
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # Lectura del joystick y actualizacion de la posicion del cursor
    a0_val = a0.read()
    a1_val = a1.read()

    if a0_val is not None:
        if a0.read() < 0.3 and posx >= 0:
            posx -= 5
        elif a0.read() > 0.8 and posx <= WIDTH - SIDE:
            posx += 5

    if a1_val is not None:
        if a1.read() < 0.3 and posy <= HEIGHT - SIDE:
            posy += 5
        elif a1.read() > 0.8 and posy >= 0:
            posy -= 5
        
    # Actualizacion de la pantalla del juego
    screen.fill(WHITE)        
    pygame.draw.rect(screen, RED, Rect((posx, posy), (SIDE, SIDE)))
    pygame.display.update()  
    clock.tick(25)
    
board.close()